# Ўзбекистонда Эвапотранспирация

In [ ]:
import pandas, glob, penmon.eto as pm

def calculate_eto(col):
    station = pm.Station(col["latitude"], col["altitude"])
    day = station.get_day(col["date"].strftime("%Y-%m-%d"))
    day.temp_min = col["temp_min"]
    day.temp_max = col["temp_max"]
    day.wind_speed=col["wind_speed"]
    day.humidity_mean=round(col["humidity_mean"] * 100, 0)
    day.radiation_s = col["solar_radiation"]
    return day.eto()
    
all_csv_files = glob.glob("data/*.csv")
all_dataframes = []
count = 0
for csv_file in all_csv_files:
    df = pandas.read_csv(csv_file, header=0,
                         names=["date", "longitude", "latitude", "altitude", "temp_max",
                               "temp_min", "precip", "wind_speed","humidity_mean", "solar_radiation", None], 
                        parse_dates=[0], index_col=None)
    df["eto"] = df.apply(calculate_eto, axis=1)
    path=csv_file.split('\\')
    df.to_csv("data/eto"+path[1], index=False, 
              columns=["date", "latitude", "longitude", "altitude", "temp_min", "temp_max",
                      "wind_speed", "humidity_mean", "solar_radiation", "eto"])
    count += 1
print("Processed ", count, "files")

In [179]:
all_dataframes[0].drop([None, "precip"], axis=1).head()

,date,longitude,latitude,altitude,temp_max,temp_min,wind_speed,humidity_mean,solar_radiation,eto
0,1979-01-01,64.375,37.311298,263,17.353,8.399,3.303726,0.503025,9.721242,2.54
1,1979-01-02,64.375,37.311298,263,17.149,8.650,3.160064,0.503016,9.331872,2.47
2,1979-01-03,64.375,37.311298,263,18.110,8.583,3.171808,0.465327,9.766629,2.67
3,1979-01-04,64.375,37.311298,263,19.278,9.860,3.767108,0.419957,9.840988,3.32
4,1979-01-05,64.375,37.311298,263,18.654,9.995,4.415612,0.382099,10.184175,3.77


In [180]:
all_dataframes[0].groupby(["date"]).mean()

,longitude,latitude,altitude,temp_max,temp_min,precip,wind_speed,humidity_mean,solar_radiation,NaN,eto
date,,,,,,,,,,,
1979-01-01,64.375,37.311298,263,17.353,8.399,0.0,3.303726,0.503025,9.721242,NaN,2.54
1979-01-02,64.375,37.311298,263,17.149,8.650,0.0,3.160064,0.503016,9.331872,NaN,2.47
1979-01-03,64.375,37.311298,263,18.110,8.583,0.0,3.171808,0.465327,9.766629,NaN,2.67
1979-01-04,64.375,37.311298,263,19.278,9.860,0.0,3.767108,0.419957,9.840988,NaN,3.32
1979-01-05,64.375,37.311298,263,18.654,9.995,0.0,4.415612,0.382099,10.184175,NaN,3.77
...,...,...,...,...,...,...,...,...,...,...,...
2014-07-27,64.375,37.311298,263,45.903,16.414,0.0,1.672771,0.115369,27.927679,NaN,8.44
2014-07-28,64.375,37.311298,263,44.720,19.405,0.0,3.343331,0.128442,27.698367,NaN,11.46
2014-07-29,64.375,37.311298,263,37.282,21.294,0.0,3.614047,0.094313,27.452272,NaN,10.46


In [142]:
all_dataframes[0].drop(["longitude", None, "precip"], axis=1).head()

,date,latitude,altitude,temp_max,temp_min,wind_speed,humidity_mean,solar_radiation,eto
0,1/1/1979,37.311298,263,17.353,8.399,3.303726,0.503025,9.721242,2.54
1,1/2/1979,37.311298,263,17.149,8.650,3.160064,0.503016,9.331872,2.47
2,1/3/1979,37.311298,263,18.110,8.583,3.171808,0.465327,9.766629,2.67
3,1/4/1979,37.311298,263,19.278,9.860,3.767108,0.419957,9.840988,3.32
4,1/5/1979,37.311298,263,18.654,9.995,4.415612,0.382099,10.184175,3.77


In [140]:
all_dataframes[0].isnull().sum()

date                   0
longitude              0
latitude               0
altitude               0
temp_max               0
temp_min               0
precip                 0
wind_speed             0
humidity_mean          0
solar_radiation        0
NaN                12928
eto                    0
dtype: int64

In [1]:
all_dataframes[0].group_by(["latitude"]).sum()

NameError: name 'all_dataframes' is not defined